# AOC2022

## Day 19 / Part 1 / Not Enough Minerals

Problem Description: https://adventofcode.com/2022/day/19

Input: [Example](aoc2022_day19_example.txt)

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
"""Solution for AOC2022, day 19, part 1."""
from enum import IntEnum
import json
import logging
import re
import sys
from pulp import LpMaximize, LpProblem, LpStatus, LpVariable, LpInteger
from pulp import PULP_CBC_CMD

LOGGER = logging.getLogger(__name__)

# show/hide debug logs
SHOW_DEBUG_LOG = False
# set input file
INPUT_FILE = "aoc2022_day19_example.txt"

In [3]:
TIMEFRAME = 24
RESOURCE_TYPE_TO_MAXIMIZE = "GEODE"

In [4]:
class ResourceType(IntEnum):
    """Resource type enumeration."""
    ORE = 0
    CLAY = 1
    OBSIDIAN = 2
    GEODE = 3


class Model:
    """Model class with helper functions for the underlying LP problem."""

    resource_types = [
        int(r) for r in ResourceType
    ]
    """Resource types."""
    robot_types = [
        # 0=ore-robot; 1=clay-robot, 2=obsidian-robot, 3=geode-robot
        int(r) for r in ResourceType
    ]
    """Robot types."""
    time_points = []
    """Time points."""

    robot_build_matrix = None
    """
    Matrix indicating wherever a robot of a specific type was built
    at a specific time point.
    """
    robot_cost_matrix = None
    """
    Matrix holding the costs (required resources)
    to build a specific robot type.
    """

    lp_problem = None
    """The LP problem."""

    def __init__(self, robot_cost_matrix, obj_resource_type, obj_minute):
        self.time_points = list(range(obj_minute + 1))
        self.robot_build_matrix = LpVariable.dicts(
            name="B",
            indices=(self.robot_types, self.time_points),
            lowBound=0, upBound=1, cat=LpInteger
        )
        # initialize robot_build_matrix for t=0
        for resource_type in self.resource_types:
            self.robot_build_matrix[resource_type][0] = 0
        # we start with a single ore robot at t=0
        self.robot_build_matrix[0][0] = 1

        self.robot_cost_matrix = robot_cost_matrix
        assert len(self.robot_cost_matrix) == len(self.robot_types)
        for robot_type in self.robot_types:
            assert len(
                self.robot_cost_matrix[robot_type]
            ) == len(self.resource_types)

        self.lp_problem = LpProblem(
            name="not_enough_minerals", sense=LpMaximize
        )

        for time_point in self.time_points:
            for resource_type in self.resource_types:
                self.lp_problem += \
                    self.get_resource_constraints(resource_type, time_point)
            self.lp_problem += self.get_robot_constraints(time_point)

        # maximize the amount of geode resources at the beginning of t=25min
        # (== at the end of t=24min)
        obj_func = self.get_resources(int(obj_resource_type), obj_minute+1)
        self.lp_problem += obj_func

    def get_robots(self, robot_type, time_point):
        """Get robots at the end of time point time_point."""
        assert robot_type in self.robot_types
        assert time_point >= 0
        expr = 0
        for t_i in range(time_point+1):
            expr += self.robot_build_matrix[robot_type][t_i]
        return expr

    def get_resources(self, resource_type, time_point):
        """Get resources at the beginning of time point time_point."""
        assert resource_type in self.resource_types
        assert time_point >= 0
        expr = 0
        for t_i in range(time_point):
            expr += (
                (time_point-t_i-1) *
                self.robot_build_matrix[resource_type][t_i]
            )
        return expr

    def get_resource_constraints(self, resource_type, time_point):
        """
        Get resource constraints for a given type and time point time_point.
        """
        assert resource_type in self.resource_types
        assert time_point >= 0
        # assert available resources >= total robot production costs
        total_robot_costs = 0
        for robot_type in self.robot_types:
            total_robot_costs += (
                (
                    self.get_robots(robot_type, time_point) -
                    (
                        # the 1st ore robot is for free
                        1 if robot_type == 0 else 0
                    )
                )
                * self.robot_cost_matrix[robot_type][resource_type]
            )
        return (
            self.get_resources(resource_type, time_point) >= total_robot_costs
        )

    def get_robot_constraints(self, time_point):
        """Get robot constraints for a time point time_point."""
        assert time_point >= 0
        # assert max one robot is created per time point
        robots_build_count = 0
        for resource_type in self.resource_types:
            robots_build_count += \
                self.robot_build_matrix[resource_type][time_point]
        return robots_build_count <= 1

    def get_details(self):
        """Get details about the model."""
        details = {}
        details["status"] = \
            f"{self.lp_problem.status}, {LpStatus[self.lp_problem.status]}"
        details["objective"] = self.lp_problem.objective.value()
        details["variables"] = [
            f"{var.name}: {var.value()}" for var in self.lp_problem.variables()
        ]
        details["constraints"] = [
            f"{name}: {constraint.value()}"
            for name, constraint in self.lp_problem.constraints.items()
        ]
        return details

    def check_constraints(self):
        """Check if all constraints are fulfilled."""
        # replace the variables of the robot_build_matrix with values
        for var in self.lp_problem.variables():
            _, resource_type, time_point = var.name.split("_")
            resource_type = int(resource_type)
            time_point = int(time_point)
            self.robot_build_matrix[resource_type][time_point] = var.value()

        for time_point in self.time_points:
            for resource_type in self.resource_types:
                assert self.get_resource_constraints(resource_type, time_point)
            assert self.get_robot_constraints(time_point)

    def solve(self):
        """Solve problem and return (status, objective_value)."""
        status = self.lp_problem.solve(PULP_CBC_CMD(msg=False))
        return (status, self.lp_problem.objective.value())

In [5]:
def main():
    """Main function to solve puzzle."""
    with open(INPUT_FILE, encoding="utf-8") as file_obj:
        quality_levels = []
        for line in [line.rstrip() for line in file_obj.readlines()]:
            (
                blueprint_id,
                ore_robot_costs_ore, clay_robot_costs_ore,
                obsidian_robot_costs_ore, obsidian_robot_costs_clay,
                geode_robot_costs_ore, geode_robot_costs_obsidian
            ) = map(
                int,
                re.search(
                    r"Blueprint (\d+): Each ore robot costs (\d+) ore. " +
                    r"Each clay robot costs (\d+) ore. " +
                    r"Each obsidian robot costs (\d+) ore and (\d+) clay. " +
                    r"Each geode robot costs (\d+) ore and (\d+) obsidian.",
                    line
                ).groups()
            )

            LOGGER.debug("processing blueprint #%s...", blueprint_id)

            robot_cost_matrix = [
                [ore_robot_costs_ore, 0, 0, 0],
                [clay_robot_costs_ore, 0, 0, 0],
                [obsidian_robot_costs_ore, obsidian_robot_costs_clay, 0, 0],
                [geode_robot_costs_ore, 0, geode_robot_costs_obsidian, 0]
            ]

            model = Model(
                robot_cost_matrix,
                obj_resource_type=ResourceType[RESOURCE_TYPE_TO_MAXIMIZE],
                obj_minute=TIMEFRAME
            )
            _, obj_value = model.solve()
            model.check_constraints()

            LOGGER.debug("  objective value: %s", str(obj_value))
            LOGGER.debug(
                "  lp problem details: %s",
                json.dumps(model.get_details(), indent=2).replace("\n", "\n  ")
            )
            LOGGER.debug("")

            quality_levels.append(int(blueprint_id) * obj_value)

    print(f"solution: {sum(quality_levels)}")

In [6]:
if __name__ == "__main__":
    LOGGER.setLevel(logging.DEBUG if SHOW_DEBUG_LOG else logging.INFO)
    log_formatter = logging.Formatter("%(message)s")
    log_handler = logging.StreamHandler(sys.stdout)
    log_handler.setFormatter(log_formatter)
    LOGGER.addHandler(log_handler)
    main()

solution: 33.0
